In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import re
import requests

In [3]:
proxy = "http://192.198.126.179:7222"

In [4]:
job_link = []
Category = []
def adzuna_extractor():#this functions extracts the job links from the particular websie
    try:
        for i in range(0,20): # iterates upto 20 pages
            url_next='https://www.adzuna.co.uk/jobs/search?loc=86383&q=data%20science&page='+str(i)
            w1=requests.get(url_next,proxy) # to access the url
            soups=BeautifulSoup(w1.content,'html.parser') 
            soup = soups.findAll(class_="text-base md:text-2xl text-adzuna-green-500 hover:underline", href=True) #find all the data from html of respective class
            for item in soup: # iterates through the items which has href links
                if item['href'].startswith('https://www.adzuna.co.uk/jobs'):
                    job_link.append(item['href']) # adds the iterted data to the empty list
                    Category.append('Data Science')
    except:
        pass         
    return job_link,Category

In [49]:
job_link,Category = adzuna_extractor()

In [50]:
Job_posting_df=pd.DataFrame({"Category":Category,"Job_Posting_link":job_link})
Job_posting_df.to_csv('Data_science_adzuna_links.csv', index=False)

In [ ]:
Job_posting_df_2=pd.DataFrame({"Category":Category2,"Job_Posting_link":job_link2})

Job_posting_df_2.to_csv('Machine_Learning_adzuna_links.csv', index=False)

In [5]:
df = pd.read_csv("Data_science_adzuna_links.csv")

In [54]:
#for data science
links = []
def redirect_link_extractor(): # this functin extratcs the links which have been redirected to other page
    for i in range(0,len(df)):
        job_links = df['Job_Posting_link'][i]
        direct_link = requests.get(job_links,proxy)
        so = BeautifulSoup(direct_link.content,'html.parser')
        redirect_link = so.find('a',href=True).get('href')
        links.append(redirect_link)
    return links


In [ ]:
links = redirect_link_extractor()

In [10]:
DS_links = pd.read_csv('Data_science_adzuna_redirect_links.csv')
ML_links = pd.read_csv('Machine_Learning_adzuna_redirect_links.csv')

In [24]:
def adzuna_details_extractor(df): # this function extracts the data from the particular job link
    adzuna_df = pd.DataFrame()
    for i in range(0,len(df)): #it iterates through the links upto the extracted links length
        try:
            Job_Posting_link=DS_links['redirect_links'][20] #iterates through the links in the dataframe
            Category=DS_links['Category'][21]
            w1 = requests.get(Job_Posting_link)
            soups1=BeautifulSoup(w1.content,'html.parser')
            try:
                Location=soups1.find(class_='location icon').text #extracts the text data from the respective class
                pass
            except:
                Location=None
            try:
                company = soups1.find(class_ = 'company icon').text
                pass
            except:
                company=None
            try:
                title = soups1.find(class_ = 'brand-font').text
                pass
            except:
                title=None
            try:
                salary = soups1.find(class_ ='salary icon').text
                pass
            except:
                salary=None
            try:
                description = soups1.find(class_='job-description').text
            except:
                description=None
            description = []
            for i in description:
                description.append(i)
            adzuna=pd.DataFrame({'Category':Category,'Job_Posting_link':Job_Posting_link,'Job_title':title,'Company_name':company,'salary':salary,'Location':Location,'Description':[description]},index=[0])
            # print(simplyhired1)
            adzuna_df=adzuna_df.append(adzuna)
        except Exception as e: # raises an exception if something goes wrong in the time of extraction
            print(e)   
    return adzuna_df

In [ ]:
ds = adzuna_details_extractor()